# Comparison and features of Deuces package implementation
Comparing the speed of poker hand evaluation and probability computation using Monte Carlo simulation techniques. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import time
import itertools

sns.set_theme()

### Speed of previous implementation

In [2]:
import pokerUtils
from pokerUtils import score_hand, Deck, runSimulation, computeProbability

In [3]:
%%timeit
simOut = runSimulation(1000,hand=['H13','C13'])
# print('probability of winning with pocket kings (n=1,000): {}%'.format(
#     round(np.sum(simOut)/len(simOut)*100,4)))

1.34 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To compute a probability using only the hole cards takes the old implementation 1.32 s per evaluation. 

In [ ]:
%%timeit
simOut = runSimulation(1000,hand=['H13','C13'],flop=['D13'])
# print('probability of winning with pocket kings (n=1,000): {}%'.format(
#     round(np.sum(simOut)/len(simOut)*100,4)))

The speed of the evaluation should not be effected by the input of other cards that would be theoretically dealt in other rounds, this is because these cards would be simulated internally if they were not entered. Thus, the combinations of the hold cards with the board is still the same level of rigor regardless of the inputs for the dealt rounds. 

### Speed using Deuces Implementation

In [ ]:
import deucesCode
from deucesCode import runSims, Evaluator, Deck

evalObj = Evaluator()

In [ ]:
%%timeit
simOut = runSims(evalObj,1000,hole=['H13','C13'])

In [ ]:
%%timeit
simOut = runSims(evalObj,1000,hole=['H13','C13'],flop=['D13'])

### Testing of individual scoring functions

Old method
> Note, in the old method for a 7 card (board + hole cards) evaluation, all the combinations of the 7 cards had to be computed and then each of those combinations scored.

In [ ]:
def combinations(arr, n):
    arr = np.asarray(arr)
    t = np.dtype([('', arr.dtype)]*n)
    result = np.fromiter(itertools.combinations(arr, n), t)
    return result.view(arr.dtype).reshape(-1, n)

In [ ]:
board = ['H13','C13','D13','H2','C10']
hand = ['H10','D7']

In [ ]:
%%timeit
combos = combinations(hand+board,5)

scores = [score_hand(c,scoreOnly=True) for c in combos]

Note, Deuces testing here is being used with 7 cards total but it can score it in a single call, the old method requires you to compile the combinations between the cards into 5 card hands from the original 7. Thus, this computation should be included in the timing of the approach because it is required in practice.

Deuces Testing

In [ ]:
%%timeit
[deucesCode.deucesFormatConvert(c) for c in hand]

Above is the timing of the format conversion.

In [ ]:
from deucesCode import Deck
deck = Deck()
aEval = Evaluator()

hand = deck.draw(2)
board = deck.draw(5)

In [ ]:
%%timeit
aEval.evaluate(hand,board)

Above is the timing of the same 7 card evaluation from Deuces instead of the old methodology. 

### Conclusions
The Deuces implementation provides significantly faster scoring of poker hands than the previous implementation. When comparing the evaluation of 7 cards using the same methodology Deuces is roughly 200 times faster. 

In terms of the Monte Carlo probability models, the difference is milliseconds to seconds (deuces, old method respectively) so thousands of times faster. 

This will allow for the evaluation and creation of the probability factors of the newly created poker game dataset. This dataset has approximately 42,000 games, roughly assuming that each game has only one player and one game to quantify, Deuces will allow the calculations to occur in roughly 3-4 hours with its millisecond computation time where previously it would have taken over a day. 